In [2]:
! uv pip install langchain google-generativeai rapidocr-onnxruntime python-dotenv langchain-community

Using Python 3.12.3 environment at: C:\mlops\.venv
Resolved 86 packages in 6.61s
Prepared 51 packages in 52.12s
Installed 82 packages in 7.02s
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.2
 + aiosignal==1.4.0
 + annotated-types==0.7.0
 + anyio==4.11.0
 + attrs==25.4.0
 + cachetools==6.2.1
 + certifi==2025.10.5
 + charset-normalizer==3.4.4
 + coloredlogs==15.0.1
 + dataclasses-json==0.6.7
 + flatbuffers==25.9.23
 + frozenlist==1.8.0
 + google-ai-generativelanguage==0.6.15
 + google-api-core==2.28.1
 + google-api-python-client==2.187.0
 + google-auth==2.43.0
 + google-auth-httplib2==0.2.1
 + google-generativeai==0.8.5
 + googleapis-common-protos==1.72.0
 + greenlet==3.2.4
 + grpcio==1.76.0
 + grpcio-status==1.71.2
 + h11==0.16.0
 + httpcore==1.0.9
 + httplib2==0.31.0
 + httpx==0.28.1
 + httpx-sse==0.4.3
 + humanfriendly==10.0
 + idna==3.11
 + jsonpatch==1.33
 + jsonpointer==3.0.0
 + langchain==1.0.5
 + langchain-classic==1.0.0
 + langchain-community==0.4.1
 + langchain-core==1.0.4
 + lan

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")

# Data Ingestion

In [6]:
from langchain_community.document_loaders import TextLoader

In [13]:
loader = TextLoader(r'C:\mlops\data\data.txt', encoding='utf8')
documents = loader.load()

In [14]:
documents[0].page_content[:500]

'Lufthansa flies back to profit\n\nGerman airline Lufthansa has returned to profit in 2004 after posting huge losses in 2003.\n\nIn a preliminary report, the airline announced net profits of 400m euros ($527.61m; Â£274.73m), compared with a loss of 984m euros in 2003. Operating profits were at 380m euros, ten times more than in 2003. Lufthansa was hit in 2003 by tough competition and a dip in demand following the Iraq war and the killer SARS virus. It was also hit by troubles at its US catering busin'

# Now divide it into chunks

In [19]:
! uv pip install langchain-text-splitters

Using Python 3.12.3 environment at: C:\mlops\.venv
Audited 1 package in 733ms


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [23]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)

In [24]:
text_chunks = text_splitter.split_documents(documents)

In [25]:
text_chunks

[Document(metadata={'source': 'C:\\mlops\\data\\data.txt'}, page_content='Lufthansa flies back to profit\n\nGerman airline Lufthansa has returned to profit in 2004 after posting huge losses in 2003.'),
 Document(metadata={'source': 'C:\\mlops\\data\\data.txt'}, page_content='In a preliminary report, the airline announced net profits of 400m euros ($527.61m; Â£274.73m), compared with a loss of 984m euros in 2003. Operating profits were at 380m euros, ten times more than'),
 Document(metadata={'source': 'C:\\mlops\\data\\data.txt'}, page_content='ten times more than in 2003. Lufthansa was hit in 2003 by tough competition and a dip in demand following the Iraq war and the killer SARS virus. It was also hit by troubles at its US catering'),
 Document(metadata={'source': 'C:\\mlops\\data\\data.txt'}, page_content='at its US catering business. Last year, Lufthansa showed signs of recovery even as some European and US airlines were teetering on the brink of bankruptcy. The board of Lufthansa 

In [26]:
! uv pip install faiss-cpu

Using Python 3.12.3 environment at: C:\mlops\.venv
Resolved 3 packages in 1.40s
Prepared 1 package in 13.92s
Installed 1 package in 404ms
 + faiss-cpu==1.12.0


In [40]:
! uv pip install sentence-transformers

Using Python 3.12.3 environment at: C:\mlops\.venv
Resolved 31 packages in 2.75s
Prepared 14 packages in 2m 23s
Installed 17 packages in 18.57s
 + filelock==3.20.0
 + fsspec==2025.10.0
 + huggingface-hub==0.36.0
 + jinja2==3.1.6
 + joblib==1.5.2
 + markupsafe==3.0.3
 + networkx==3.5
 + regex==2025.11.3
 + safetensors==0.6.2
 + scikit-learn==1.7.2
 + scipy==1.16.3
 + sentence-transformers==5.1.2
 + setuptools==80.9.0
 + threadpoolctl==3.6.0
 + tokenizers==0.22.1
 + torch==2.9.0
 + transformers==4.57.1


In [41]:
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS

c:\mlops\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
from langchain_core.embeddings import Embeddings
from sentence_transformers import SentenceTransformer

class HFEmbeddings(Embeddings):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)

    def embed_query(self, text: str):
        return self.model.encode(text).tolist()

    def embed_documents(self, texts: list[str]):
        return self.model.encode(texts).tolist()


In [47]:
embedder = HFEmbeddings()
faiss_store = FAISS.from_texts(
    [c.page_content for c in text_chunks],
    embedder
)
